In [3]:
# default_exp datastructure.matrix.sparse

%reload_ext autoreload
%autoreload 2

# 稀疏矩阵
https://cloud.tencent.com/developer/article/1574832
## 稀疏矩阵的定义：

具有少量非零项的矩阵（在矩阵中，若数值0的元素数目远多于非0元素的数目，并且非0元素分布没有规律时，）则称该矩阵为稀疏矩阵；相反，为稠密矩阵。非零元素的总数比上矩阵所有元素的总数为矩阵的稠密度。

稀疏矩阵的两个动机：稀疏矩阵通常具有很大的维度，有时甚大到整个矩阵（零元素）与可用内存不想适应；另一个动机是避免零矩阵元素的运算具有更好的性能。

## 稀疏矩阵的格式

存储矩阵的一般方法是采用二维数组，其优点是可以随机地访问每一个元素，因而能够容易实现矩阵的各种运算。对于稀疏矩阵，采用二维数组的存储方法既浪费大量的存储单元来存放零元素，又要在运算中浪费大量的时间来进行零元素的无效运算。因此必须考虑对稀疏矩阵进行压缩存储（只存储非零元素）。

Scipy.sparse模块提供了许多来自于稀疏矩阵的不同存储格式。这里仅描述最为重要的格式CSR、CSC和LIL。

* CSR、CSC是用于矩阵-矩阵和矩阵-向量运算的有效格式，
* LIL格式用于生成和更改稀疏矩阵。

Python不能自动创建稀疏矩阵，所以要用scipy中特殊的命令来得到稀疏矩阵。

### 压缩稀疏行（CSR，Compressed Sparse Row）：或csr_matrix  按行对矩阵进行压缩的。

   CSR使用了三个数组，分别为数值、行偏移（表示某一行的第一个元素在数值里面的起始偏移位置，在行偏移的最后补上矩阵总的元素个数）、列号。CSR是一种编码的方式

一维数组data（数值）:有序地存储了所有的非零值，它具有与非零元素同样多数量的元素，通常由变量nnz表示。

一维数组indptr（行偏移量）：包含了证书使得indptr[i]是data中元素的索引，它是行i中的第一个非零元素。如果整个行i为零，则indptr[i]==indptr[i+1]

如初始矩阵有m行，则len(indptr)==m+1

一维数组Indices（列号:）: 其使用如下方式包含列索引信息:indices[indptr[i]:indptr[i+1]]是一个具有行i中非零元素的列索引的整数数组。Len(indice)==len(data)==nnz

备注：列索引表示数值所在的列号，从0开始。

 数组data：包含矩阵中的非零元素，以行优先的形式保存。

 行偏移：CSR中行索引被压缩，没有行索引，这里用行偏移表示行索引。
![](img/sparsem01.png)

如上图所示：data=(1,7,2,8,5,3,9,6,4)

            Indices=(0,1,1,2,0,2,3,1,3)    #列索引

            Indptr=(0,2,4,7,9)  #行偏移（表示某一行的第一个元素在数值里面的起始偏移位置，在行偏移的最后补上矩阵总的元素个数）

In [18]:
from scipy import sparse as sp
import numpy as np
import pandas as pd

# 生成稀疏矩阵

## 稀疏列矩阵CSR（Compressed Sparse Row）

In [ ]:
sparse.csr_matrix()

In [8]:
indptr = np.array([0, 2, 3, 6])

indices = np.array([0, 2, 2, 0, 1, 2])

data = np.array([1, 2, 3, 4, 5, 6])

A=sp.csr_matrix((data, indices, indptr), shape=(3, 3)) #生成CSR格式的矩阵
A

<3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [9]:
A.toarray()

array([[1, 0, 2],
       [0, 0, 3],
       [4, 5, 6]])

解析：第i行的列索引存储在indices[indptr[i]:indptr[i+1]]中,对应的值为data[indptr[i]:indptr[i+1]]。

即例如第0行的列索引为indices[0:2]=[0,2]（第i行中非零元素的列索引组成的整数数组）,值为data[0:2]=[1,2];

第1行的列索引为indices[2:3]=[2],值为data[2:3]=[3]…


### from ndarray & DataFrame

In [22]:
A=np.array([[1,0,2,0],[0,0,0,0],[3,0,0,0],[1,0,0,4]])

AS=sp.csr_matrix(A)
AS

<4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [23]:
df = pd.DataFrame(A)
AS=sp.csr_matrix(A)
AS

<4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [11]:
print(AS)

  (0, 0)	1
  (0, 2)	2
  (2, 0)	3
  (3, 0)	1
  (3, 3)	4


In [12]:
print(AS.data)  # [1 2 3 1 4]
# 行偏移（表示某一行的第一个元素在数值里面的起始偏移位置，在行偏移的最后补上矩阵总的元素个数）
print(AS.indptr)  # [0 2 2 3 5] 

print(AS.indices)  # 列索引 [0 2 0 0 3]

print(AS.nnz)    # 5

[1 2 3 1 4]
[0 2 2 3 5]
[0 2 0 0 3]
5


### 矩阵合并

In [26]:
sp.hstack([AS,AS]).toarray()

array([[1, 0, 2, 0, 1, 0, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [3, 0, 0, 0, 3, 0, 0, 0],
       [1, 0, 0, 4, 1, 0, 0, 4]], dtype=int64)

In [28]:
sp.vstack([AS,AS])

<8x4 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [27]:
sp.vstack([AS,AS]).toarray()

array([[1, 0, 2, 0],
       [0, 0, 0, 0],
       [3, 0, 0, 0],
       [1, 0, 0, 4],
       [1, 0, 2, 0],
       [0, 0, 0, 0],
       [3, 0, 0, 0],
       [1, 0, 0, 4]], dtype=int64)

## 稀疏列矩阵CSC（Compressed Sparse Column）,
用于CSC格式的类型为：csc_matrix  按列对矩阵进行压缩的。

 与CSR格式相比唯一的不同点是indptr和indices数组的定义，该定义与列有关。
 
Advantages of the CSC format
    - efficient arithmetic operations CSC + CSC, CSC * CSC, etc.
    - efficient column slicing
    - fast matrix vector products (CSR, BSR may be faster)

Disadvantages of the CSC format
  - slow row slicing operations (consider CSR)
  - changes to the sparsity structure are expensive (consider LIL or DOK)

### 生成CSC矩阵
https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html#scipy.sparse.csc_matrix



    csc_matrix(D)

        with a dense matrix or rank-2 ndarray D
    csc_matrix(S)

        with another sparse matrix S (equivalent to S.tocsc())
    csc_matrix((M, N), [dtype])

        to construct an empty matrix with shape (M, N) dtype is optional, defaulting to dtype=’d’.
    csc_matrix((data, (row_ind, col_ind)), [shape=(M, N)])

        where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k].
    csc_matrix((data, indices, indptr), [shape=(M, N)])

        is the standard CSC representation where the row indices for column i are stored in indices[indptr[i]:indptr[i+1]] and their corresponding values are stored in data[indptr[i]:indptr[i+1]]. If the shape parameter is not supplied, the matrix dimensions are inferred from the index arrays.



In [13]:
A = sp.csc_matrix((2, 3))
A.toarray()

array([[0., 0., 0.],
       [0., 0., 0.]])

### 稀疏矩阵序列化和加载

In [84]:
sp.save_npz('data/t.npz', A)

In [85]:
A = sp.load_npz('data/t.npz')
A.toarray()

array([[0., 0., 0.],
       [0., 0., 0.]])

In [54]:
A[1, 2] = 5

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [44]:


A=np.array([[1,0,2,0],[0,0,0,0],[3,0,0,0],[1,0,0,4]])

AS=sp.csc_matrix(A)

print(A)
"""
[[1 0 2 0]
 [0 0 0 0]
 [3 0 0 0]
 [1 0 0 4]]
"""
print(AS)
"""
(0, 0)	1
(2, 0)	3
(3, 0)	1
(0, 2)	2
(3, 3)	4
"""
print(AS.data)  # [1 3 1 2 4]

print(AS.indptr)  # [0 3 3 4 5]

print(AS.indices)  # 行索引 3[0 2 3 0 3]

print(AS.nnz)    # 5

[[1 0 2 0]
 [0 0 0 0]
 [3 0 0 0]
 [1 0 0 4]]
  (0, 0)	1
  (2, 0)	3
  (3, 0)	1
  (0, 2)	2
  (3, 3)	4
[1 3 1 2 4]
[0 3 3 4 5]
[0 2 3 0 3]
5


## 基于行的链表格式：LIL（Row-Based Linked List Format）
LIL格式最适合切片的方法，即以LIL格式提取子矩阵，并通过插入非零元素来改变稀疏模式。

 1. 链表稀疏格式在列表数据中以行方式存储非零元素，

列表data: data[k]是行k中的非零元素的列表。如果该行中的所有元素都为0，则它包含一个空列表。

列表rows: 是在位置k包含了在行k中的非零元素列索引列表。

In [2]:
A=np.array([[1,0,2,0],[0,0,0,0],[3,0,0,0],[1,0,0,4]])

AS=sp.lil_matrix(A)
AS[1,2] = 5

In [3]:
AS.toarray()

array([[1, 0, 2, 0],
       [0, 0, 5, 0],
       [3, 0, 0, 0],
       [1, 0, 0, 4]], dtype=int64)

In [11]:
AS.data

array([list([1, 2]), list([5]), list([3]), list([1, 4])], dtype=object)

In [22]:
AS.data[0]

[1, 2]

In [23]:
AS.rows[0]

[0, 2]

In [25]:
AS.shape

(4, 4)

### 遍历lil的每一个元素

In [26]:
for r in range(AS.shape[0]):
    for c in AS.rows[r]:
        print(r,c,AS[r,c])

0 0 1
0 2 2
1 2 5
2 0 3
3 0 1
3 3 4


In [9]:
import pandas as pd

In [8]:
AS[0]

<1x4 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in List of Lists format>

In [12]:
pd.Series(AS[0].toarray()[0])

0    1
1    0
2    2
3    0
dtype: int64

In [5]:
AS[0].toarray()  # 取出某一行数据

array([[1, 0, 2, 0]], dtype=int64)

In [7]:
AS[[0,2]].toarray()  # 取出某几行数据

array([[1, 0, 2, 0],
       [3, 0, 0, 0]], dtype=int64)

In [65]:
AS.T.toarray()

array([[1, 0, 3, 1],
       [0, 0, 0, 0],
       [2, 5, 0, 0],
       [0, 0, 0, 4]], dtype=int64)

In [66]:

print(AS.data)

print(AS.rows)

print(AS.nnz) 

[list([1, 2]) list([5]) list([3]) list([1, 4])]
[list([0, 2]) list([2]) list([0]) list([0, 3])]
6


In [76]:
cc = AS.tocsc()

In [77]:
cc.data

array([1, 3, 1, 2, 5, 4], dtype=int64)

In [78]:
cc.data = cc.data + 1

In [79]:
cc.toarray()

array([[2, 0, 3, 0],
       [0, 0, 6, 0],
       [4, 0, 0, 0],
       [2, 0, 0, 5]], dtype=int64)

In [72]:
cc.data = np.log(cc.data)

In [73]:
cc.data

array([0.69314718, 1.09861229, 0.        , 0.69314718, 1.60943791,
       1.38629436])

In [75]:
AS.toarray()

array([[1, 0, 2, 0],
       [0, 0, 5, 0],
       [3, 0, 0, 0],
       [1, 0, 0, 4]], dtype=int64)

In [74]:
cc.toarray()

array([[0.69314718, 0.        , 0.69314718, 0.        ],
       [0.        , 0.        , 1.60943791, 0.        ],
       [1.09861229, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.38629436]])

In [67]:
AS.data = np.log(AS.data)

TypeError: loop of ufunc does not support argument 0 of type list which has no callable log method

In [57]:
type(AS)

scipy.sparse.lil.lil_matrix

In [58]:
AS.tocsc()

<4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Column format>

In [59]:
type(AS)

scipy.sparse.lil.lil_matrix

## sparse模块中用于创建稀疏矩阵的函数

### 对角矩阵

In [14]:
x = sparse.eye(5)
x

<5x5 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements (1 diagonals) in DIAgonal format>

In [15]:
x.toarray()

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

### 随机矩阵

In [16]:
x = sparse.random(2, 3, 0.5)  # 第3个参数表示非0元素的密度

In [17]:
x.toarray()

array([[0.        , 0.        , 0.71079621],
       [0.51861479, 0.62267008, 0.        ]])

# nb_export

In [4]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted engineering_nbdev.ipynb.
Converted index.ipynb.


In [7]:
!nbdev_build_docs

No notebooks were modified
converting /Users/luoyonggui/PycharmProjects/nbdevlib/index.ipynb to README.md
